In [1]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import cv2


2024-03-11 08:43:03.202496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("TRAINING------")
for emotion in os.listdir('train/'):
    print(str(len(os.listdir('train/' + emotion))) + ': ' + emotion + ' imgs')

print("TEST------")
for emotion in os.listdir('test/'):
    print(str(len(os.listdir('test/' + emotion))) + ': ' + emotion + ' imgs')

TRAINING------
7215: happy imgs
4830: sad imgs
4097: fear imgs
3171: surprise imgs
4965: neutral imgs
3995: angry imgs
436: disgust imgs
TEST------
1774: happy imgs
1247: sad imgs
1024: fear imgs
831: surprise imgs
1233: neutral imgs
958: angry imgs
111: disgust imgs


In [3]:
# Set image and batch size
image_size = (224,224)
batch_size = 100

# Define the directories for training and testing data
train_data = 'train/'
test_data = 'test/'

# Create the image generator for normilization
train_data_gen = ImageDataGenerator(horizontal_flip=True)
test_data_gen = ImageDataGenerator(horizontal_flip=True)

# Generate the batches of training and test data
train_generator = train_data_gen.flow_from_directory(
    train_data,
    target_size = image_size,
    batch_size = batch_size,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

test_generator = test_data_gen.flow_from_directory(
    test_data,
    target_size = image_size,
    batch_size = batch_size,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224,224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size)

Epoch 1/10
287/287 [==============================] - 4104s 14s/step - loss: 4.2696 - accuracy: 0.2875 - val_loss: 1.6523 - val_accuracy: 0.3583
Epoch 2/10
287/287 [==============================] - 3982s 14s/step - loss: 1.5769 - accuracy: 0.3883 - val_loss: 1.5606 - val_accuracy: 0.3934
Epoch 3/10
287/287 [==============================] - 3873s 13s/step - loss: 1.4367 - accuracy: 0.4501 - val_loss: 1.5150 - val_accuracy: 0.4254
Epoch 4/10
287/287 [==============================] - 3865s 13s/step - loss: 1.3173 - accuracy: 0.5032 - val_loss: 1.4484 - val_accuracy: 0.4492
Epoch 5/10
287/287 [==============================] - 3913s 14s/step - loss: 1.1992 - accuracy: 0.5563 - val_loss: 1.4619 - val_accuracy: 0.4538
Epoch 6/10
287/287 [==============================] - 4006s 14s/step - loss: 1.0734 - accuracy: 0.6066 - val_loss: 1.5375 - val_accuracy: 0.4630
Epoch 7/10
287/287 [==============================] - 3886s 14s/step - loss: 0.9486 - accuracy: 0.6617 - val_loss: 1.5919 - val_ac

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {accuracy}')

71/71 [==============================] - 299s 4s/step - loss: 1.9805 - accuracy: 0.4521
Test accuracy: 0.45211267471313477


In [8]:
model.save('output/emotion_detection.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
# Get the name of the last convolutional layer
last_conv_layer_name = 'conv2d_2'

# Load a sample image from your test dataset
sample_image_path = 'test/fear/PrivateTest_623230.jpg'
img = image.load_img(sample_image_path, target_size=(224, 224), color_mode='grayscale')
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Define a function to generate CAM using OpenCV
def generate_cam(model, last_conv_layer_name, image_path, target_size=(224, 224)):
    # Load the image
    img = image.load_img(image_path, target_size=target_size, color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Extract the last convolutional layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    
    # Create a new model with the last convolutional layer as output
    cam_model = Model(inputs=model.input, outputs=last_conv_layer.output)
    
    # Get the weights of the output layer
    output_weights = model.layers[-1].get_weights()[0]
    
    # Compute the feature maps
    features = cam_model.predict(img_array)
    
    # Calculate the class activation map
    cam = np.matmul(features, output_weights)
    cam = np.sum(cam, axis=-1)
    
    # Normalize CAM
    cam /= np.max(cam)
    
    # Resize CAM to the size of the input image
    cam = cv2.resize(cam[0], target_size)
    
    # Convert CAM to heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    
    # Convert grayscale image to BGR
    img_bgr = cv2.cvtColor(np.uint8(img_array[0]), cv2.COLOR_GRAY2BGR)
    
    # Overlay the heatmap on the original image
    cam_overlay = cv2.addWeighted(heatmap, 0.5, img_bgr, 0.5, 0)
    
    return cam_overlay



# Generate CAM using OpenCV
cam_overlay = generate_cam(model, last_conv_layer_name, sample_image_path)


# Generate CAM using OpenCV
cam_overlay = generate_cam(model, last_conv_layer_name, sample_image_path)

cv2.imwrite('output/cam_fear.jpg', cam_overlay)

1/1 [==============================] - 1s 1s/step


True

In [17]:
from keras.models import load_model

# Load the trained model
model = load_model('output/emotion_detection.h5')

# Define the target size for resizing the images
target_size = (224, 224)

# Define a function to preprocess the image
def preprocess_image(image_path):
   # Read the image
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image to the target size
    resized_image = cv2.resize(gray_image, target_size)
    # Normalize the pixel values to be in the range [0, 1]
    normalized_image = resized_image / 255.0
    # Expand the dimensions of the image to match the input shape expected by the model
    expanded_image = np.expand_dims(normalized_image, axis=-1)
    # Return the preprocessed image
    return expanded_image

# Define a function to predict emotion from an image
def predict_emotion(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    # Make prediction
    predictions = model.predict(np.expand_dims(preprocessed_image, axis=0))
    # Get the predicted emotion class
    predicted_class = np.argmax(predictions)
    # Map the predicted class to the corresponding emotion label
    emotion_labels = ['happy', 'disgust', 'fear', 'angry', 'sad', 'surprised', 'neutral']
    predicted_emotion = emotion_labels[predicted_class]
    return predicted_emotion

# Test the model on an example image
image_path = 'test/happy/PrivateTest_556985.jpg'
predicted_emotion = predict_emotion(image_path)
print('Predicted Emotion:', predicted_emotion)


1/1 [==============================] - 1s 798ms/step
Predicted Emotion: happy


In [11]:
from sklearn.metrics import classification_report

# Generate predictions for the test data
y_pred = model.predict(test_generator)

# Convert the predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true class labels from the test data generator
y_true = test_generator.classes

# Compute precision, recall, and F1-score
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# Print the classification report
print(report)

72/72 [==============================] - 329s 5s/step
              precision    recall  f1-score   support

       angry       0.13      0.16      0.14       958
     disgust       0.00      0.00      0.00       111
        fear       0.15      0.13      0.14      1024
       happy       0.24      0.26      0.25      1774
     neutral       0.17      0.14      0.15      1233
         sad       0.19      0.20      0.19      1247
    surprise       0.10      0.10      0.10       831

    accuracy                           0.18      7178
   macro avg       0.14      0.14      0.14      7178
weighted avg       0.17      0.18      0.17      7178

